In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:

sgRNA_df = pd.read_csv('Indel\CIRCLE_seq.csv')
guide_df = pd.read_csv('Indel\GUIDE-Seq.csv')

print("sgRNA Dataset Overview:")
print(f"Shape: {sgRNA_df.shape}")
print(sgRNA_df.head())
print("\nGuide Dataset Overview:")
print(f"Shape: {guide_df.shape}")
print(guide_df.head())


sgRNA Dataset Overview:
Shape: (584949, 7)
                  sgRNA_seq                   off_seq  label  Read  \
0  G_AGTCCGAGCAGAAGAAGAAAGG  GGAGTCCGTGCAGAAGCAGGAAGC    0.0   0.0   
1  G_AGTCCGAGCAGAAGAAGAAAGG  CAAGTCCGAGAAGAAGCAGAAAAG    0.0   0.0   
2  G_AGTCCGAGCAGAAGAAGAAAGG  GAAGTCTTAGCAAAAGAAGAAAGC    0.0   0.0   
3  G_AGTCCGAGCAGAAGAAGAAAGG  GAAGTCCGAGGAGAGGAAGAAAGG    0.0   0.0   
4  G_AGTCCGAGCAGAAGAAGAAAGG  GAAGGCGGAGAAGAAGAAGAAATT    0.0   0.0   

                sgRNA_type Cell  Read_normalised  
0  GAGTCCGAGCAGAAGAAGAANGG                   0.0  
1  GAGTCCGAGCAGAAGAAGAANGG                   0.0  
2  GAGTCCGAGCAGAAGAAGAANGG                   0.0  
3  GAGTCCGAGCAGAAGAAGAANGG                   0.0  
4  GAGTCCGAGCAGAAGAAGAANGG                   0.0  

Guide Dataset Overview:
Shape: (213933, 5)
                        DNA                     crRNA  label  read  \
0  GCTGCCAGTACAGGCTCCCCCTCG  GCAGCCAGTACA_GCTCACCATGG    0.0   0.0   
1  GCTGCCAGTACAGGCTCCCCCTCG  GCAGCCAGTACAG_CTC

In [ ]:

def analyze_mismatches(guide_seq, off_target_seq):
    min_len = min(len(guide_seq), len(off_target_seq))
    
    mismatches = 0
    positions = []
    
    for i in range(min_len):
        if i < len(guide_seq) and i < len(off_target_seq):
            if guide_seq[i] != off_target_seq[i] and guide_seq[i] != '_' and off_target_seq[i] != '_':
                mismatches += 1
                positions.append(i)
    
    return mismatches, positions

def preprocess_sgRNA_data(df):
    df['sgRNA_clean'] = df['sgRNA_seq'].str.replace('G_', '')
    
    mismatch_data = [analyze_mismatches(row['sgRNA_clean'], row['off_seq']) 
                     for _, row in df.iterrows()]
    
    df['mismatch_count'] = [data[0] if data is not None else None for data in mismatch_data]
    df['mismatch_positions'] = [data[1] if data is not None else None for data in mismatch_data]
    
    df['GC_content'] = df['sgRNA_clean'].apply(
        lambda x: (x.count('G') + x.count('C')) / len(x) if x else None
    )
    
    return df

def preprocess_guide_data(df):
    df['DNA_length'] = df['DNA'].apply(len)
    df['crRNA_length'] = df['crRNA'].apply(len)
    
    df['pair_split'] = df['pair'].str.split('|')
    df['guide_from_pair'] = df['pair_split'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
    df['target_from_pair'] = df['pair_split'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
    
    mismatch_data = [analyze_mismatches(row['crRNA'], row['DNA']) 
                     for _, row in df.iterrows()]
    
    df['mismatch_count'] = [data[0] if data is not None else None for data in mismatch_data]
    df['mismatch_positions'] = [data[1] if data is not None else None for data in mismatch_data]
    
    return df

sgRNA_processed = preprocess_sgRNA_data(sgRNA_df)
guide_processed = preprocess_guide_data(guide_df)

def plot_mismatch_heatmap(df, sequence_col, title):
    max_length = df[sequence_col].apply(len).max()
    heatmap_data = np.zeros((len(df), max_length))
    
    for i, row in df.iterrows():
        positions = row['mismatch_positions']
        if positions is not None:
            for pos in positions:
                if pos < max_length:
                    heatmap_data[i, pos] = 1
    
    plt.figure(figsize=(12, 8))
    sns.heatmap(heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Mismatch'})
    plt.title(title)
    plt.xlabel('Position in Sequence')
    plt.ylabel('Sequence Index')
    plt.tight_layout()
    plt.savefig(f"output\{title.replace(' ', '_')}.png")
    plt.close()

def calculate_edit_distance(seq1, seq2):
    if not seq1 or not seq2:
        return None
    
    seq1 = seq1.replace('_', '')
    seq2 = seq2.replace('_', '')
    
    m, n = len(seq1), len(seq2)
    dp = [[0 for _ in range(n+1)] for _ in range(m+1)]
    
    for i in range(m+1):
        dp[i][0] = i
    for j in range(n+1):
        dp[0][j] = j
    
    for i in range(1, m+1):
        for j in range(1, n+1):
            if seq1[i-1] == seq2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])
    
    return dp[m][n]


In [4]:

sgRNA_processed['edit_distance'] = [
    calculate_edit_distance(row['sgRNA_clean'], row['off_seq']) 
    for _, row in sgRNA_processed.iterrows()
]

guide_processed['edit_distance'] = [
    calculate_edit_distance(row['crRNA'], row['DNA']) 
    for _, row in guide_processed.iterrows()
]

try:
    plot_mismatch_heatmap(sgRNA_processed, 'sgRNA_clean', 'Mismatch Positions in sgRNA Dataset')
    plot_mismatch_heatmap(guide_processed, 'crRNA', 'Mismatch Positions in Guide Dataset')
except Exception as e:
    print(f"Error generating heatmaps: {e}")
    print("Continuing with analysis...")

sgRNA_processed['label_binary'] = (sgRNA_processed['label'] > 0.1).astype(int)

feature_cols = ['mismatch_count', 'GC_content', 'edit_distance']
X = sgRNA_processed[feature_cols].dropna()
y = sgRNA_processed.loc[X.index, 'label_binary']

if len(X) < 10:
    print("\nNot enough data for model training. Skipping this step.")
else:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    print("\nModel Performance:")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print("\nFeature Importance:")
    for i, feature in enumerate(feature_cols):
        print(f"{feature}: {model.feature_importances_[i]:.4f}")



Model Performance:
Accuracy: 0.9874262757500641

Feature Importance:
mismatch_count: 0.3049
GC_content: 0.2097
edit_distance: 0.4854


In [5]:

print("\nData Analysis Summary:")
print(f"Total sgRNA sequences: {len(sgRNA_processed)}")
print(f"Total guide sequences: {len(guide_processed)}")
print(f"Average mismatch count in sgRNA dataset: {sgRNA_processed['mismatch_count'].mean():.2f}")
print(f"Average mismatch count in guide dataset: {guide_processed['mismatch_count'].mean():.2f}")
print(f"Average GC content in sgRNA sequences: {sgRNA_processed['GC_content'].mean():.2f}")

try:
    plt.figure(figsize=(10, 6))
    sns.histplot(sgRNA_processed['mismatch_count'].dropna(), bins=10, kde=True)
    plt.title('Distribution of Mismatches in sgRNA Dataset')
    plt.xlabel('Number of Mismatches')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig("output\sgRNA_mismatch_distribution.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    sns.histplot(sgRNA_processed['GC_content'].dropna(), bins=10, kde=True)
    plt.title('GC Content Distribution in sgRNA Sequences')
    plt.xlabel('GC Content')
    plt.ylabel('Count')
    plt.tight_layout()
    plt.savefig("output\sgRNA_GC_content_distribution.png")
    plt.close()

    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=sgRNA_processed, x='mismatch_count', y='edit_distance')
    plt.title('Relationship Between Mismatch Count and Edit Distance')
    plt.xlabel('Number of Mismatches')
    plt.ylabel('Edit Distance')
    plt.tight_layout()
    plt.savefig("output\mismatch_vs_edit_distance.png")
    plt.close()
except Exception as e:
    print(f"Error generating plots: {e}")
    print("Continuing with analysis...")



Data Analysis Summary:
Total sgRNA sequences: 584949
Total guide sequences: 213933
Average mismatch count in sgRNA dataset: 5.46
Average mismatch count in guide dataset: 5.31
Average GC content in sgRNA sequences: 0.63


In [6]:

def count_positional_mismatches(df, seq_col, off_col):
    max_length = max(df[seq_col].apply(len).max(), df[off_col].apply(len).max())
    position_counts = [0] * max_length
    
    for _, row in df.iterrows():
        seq1 = row[seq_col]
        seq2 = row[off_col]
        
        if not isinstance(seq1, str) or not isinstance(seq2, str):
            continue
            
        seq1 = seq1.ljust(max_length, 'X')
        seq2 = seq2.ljust(max_length, 'X')
        
        for i in range(min(len(seq1), len(seq2))):
            if seq1[i] != seq2[i] and seq1[i] != '_' and seq2[i] != '_' and seq1[i] != 'X' and seq2[i] != 'X':
                position_counts[i] += 1
    
    return position_counts


In [7]:

sgRNA_positional = count_positional_mismatches(sgRNA_processed, 'sgRNA_clean', 'off_seq')
guide_positional = count_positional_mismatches(guide_processed, 'crRNA', 'DNA')

try:
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(sgRNA_positional)), sgRNA_positional)
    plt.title('Positional Distribution of Mismatches in sgRNA Dataset')
    plt.xlabel('Position in Sequence')
    plt.ylabel('Number of Mismatches')
    plt.tight_layout()
    plt.savefig("output\sgRNA_positional_mismatches.png")
    plt.close()
except Exception as e:
    print(f"Error generating positional mismatch plot: {e}")

sgRNA_processed.to_csv('output\sgRNA_processed.csv', index=False)
guide_processed.to_csv('output\guide_processed.csv', index=False)

print("\nAnalysis complete. Processed data and visualizations have been saved.")

print("\n=== CRISPR-Cas9 Algorithmic Analysis ===")
print("1. Sequence Matching Algorithm:")
print("   - Target sequences analyzed: " + str(len(sgRNA_df) + len(guide_df)))
print("   - Average edit distance: " + 
      str(np.mean([x for x in sgRNA_processed['edit_distance'].dropna()] + 
                 [x for x in guide_processed['edit_distance'].dropna()])))

print("\n2. Pattern Recognition Results:")
print("   - Most common mismatch position: " + 
      str(np.argmax(sgRNA_positional)) + " in sgRNA dataset")
print("   - Guide RNA average GC content: " + 
      str(sgRNA_processed['GC_content'].mean()))

print("\n3. Guide RNA Design Implications:")
print("   - Positions with high mismatch tolerance: " + 
      str([i for i, count in enumerate(sgRNA_positional) if count > np.mean(sgRNA_positional)]))
print("   - Positions with low mismatch tolerance: " + 
      str([i for i, count in enumerate(sgRNA_positional) if count < np.mean(sgRNA_positional) and count > 0]))



Analysis complete. Processed data and visualizations have been saved.

=== CRISPR-Cas9 Algorithmic Analysis ===
1. Sequence Matching Algorithm:
   - Target sequences analyzed: 798882
   - Average edit distance: 5.577237940021179

2. Pattern Recognition Results:
   - Most common mismatch position: 23 in sgRNA dataset
   - Guide RNA average GC content: 0.6336525349503405

3. Guide RNA Design Implications:
   - Positions with high mismatch tolerance: [1, 5, 8, 9, 15, 17, 19, 22, 23]
   - Positions with low mismatch tolerance: [0, 2, 3, 4, 6, 7, 10, 11, 12, 13, 14, 16, 18, 20, 21]


In [ ]:

def predict_offtarget_likelihood(guide_seq, target_seq):
    """
    A simple algorithm to predict off-target likelihood
    based on mismatch patterns and positions
    """
    mismatches, positions = analyze_mismatches(guide_seq, target_seq)
    
    if mismatches is None:
        return "Cannot analyze sequences of different lengths"
    
    base_score = 100 - (mismatches * 20)
    
    position_penalty = 0
    seed_region_start = max(0, len(guide_seq) - 10)
    
    for pos in positions:
        if pos >= seed_region_start:
            position_penalty += 10  # Higher penalty for seed region
        else:
            position_penalty += 5   # Lower penalty elsewhere
    
    final_score = max(0, base_score - position_penalty)
    if final_score > 70:
        return f"High off-target likelihood ({final_score}%)"
    elif final_score > 40:
        return f"Medium off-target likelihood ({final_score}%)"
    else:
        return f"Low off-target likelihood ({final_score}%)"


In [ ]:


print("\n4. Off-target Prediction Examples:")
if len(sgRNA_processed) > 0:
    example_guide = sgRNA_processed.iloc[0]['sgRNA_clean']
    example_target = sgRNA_processed.iloc[0]['off_seq']
    print(f"   Guide: {example_guide}")
    print(f"   Target: {example_target}")
    print(f"   Prediction: {predict_offtarget_likelihood(example_guide, example_target)}")

print("\nThis analysis demonstrates how algorithmic principles can be applied to CRISPR guide RNA design")
print("and off-target prediction, similar to search algorithms and pattern matching in computer science.")


4. Off-target Prediction Examples:
   Guide: AGTCCGAGCAGAAGAAGAAAGG
   Target: GGAGTCCGTGCAGAAGCAGGAAGC
   Prediction: Low off-target likelihood (0%)

This analysis demonstrates how algorithmic principles can be applied to CRISPR guide RNA design
and off-target prediction, similar to search algorithms and pattern matching in computer science.
